# Realms2Vec

© Yuriy Guts, 2016

Using only the raw text of [several Forgotten Realms novels](https://en.wikipedia.org/wiki/List_of_Forgotten_Realms_novels), we'll derive and explore the semantic properties of their words.

Credits for this code go to [Yuriy Guts](https://github.com/YuriyGuts/). I've merely run it on Forgotten Realms novels corpus.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re

In [3]:
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Set up logging**

In [5]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

**Download NLTK tokenizer models (only the first time)**

In [6]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Prepare Corpus

**Load books from files**

In [7]:
book_filenames = sorted(glob.glob("data/*.txt"))

In [8]:
print("Found books:")
book_filenames

Found books:


['data/01. Dark Elf - Homeland.txt',
 'data/01. Parched Sea, The.txt',
 'data/02. Dark Elf - Exile.txt',
 'data/02. Elfshadow.txt',
 'data/03. Dark Elf - Sojourn.txt',
 'data/03. Red Magic.txt',
 'data/04. Icewind Dale - The Crystal Shard.txt',
 'data/04. Night Parade, The.txt',
 'data/05. Icewind Dale - Streams of Silver.txt',
 'data/05. Ring of Winter, The.epub.txt',
 'data/06. Crypt of the Shadowking.txt',
 "data/06. Icewind Dale - The Halfling's Gem.txt",
 'data/07. Legacy of the Drow - The Legacy.txt',
 'data/07. Soldiers of Ice.txt',
 'data/08. Elfsong.txt',
 'data/08. Legacy of the Drow - Starless Night.txt',
 'data/09. Crown of Fire.txt',
 'data/09. Legacy of the Drow - Siege of Darkness.txt',
 'data/1. A Neverwinter Novella, Part I.txt',
 'data/1. Abduction, The.txt',
 'data/1. Alabaster Staff, The.txt',
 'data/1. Archmage.txt',
 'data/1. Ascendance.txt',
 'data/1. Azure Bonds.txt',
 "data/1. Baldur's Gate.txt",
 'data/1. Blackstaff Tower.txt',
 'data/1. Blackstaff.txt',
 'dat

**Combine the books into one string**

In [9]:
corpus_raw = u""
for book_filename in book_filenames:
    print("Reading '{0}'...".format(book_filename))
    with codecs.open(book_filename, "r", "utf-8") as book_file:
        corpus_raw += book_file.read()
    print("Corpus is now {0} characters long".format(len(corpus_raw)))
    print()

Reading 'data/01. Dark Elf - Homeland.txt'...
Corpus is now 575275 characters long

Reading 'data/01. Parched Sea, The.txt'...
Corpus is now 1105935 characters long

Reading 'data/02. Dark Elf - Exile.txt'...
Corpus is now 1659679 characters long

Reading 'data/02. Elfshadow.txt'...
Corpus is now 2246584 characters long

Reading 'data/03. Dark Elf - Sojourn.txt'...
Corpus is now 2821996 characters long

Reading 'data/03. Red Magic.txt'...
Corpus is now 3401552 characters long

Reading 'data/04. Icewind Dale - The Crystal Shard.txt'...
Corpus is now 4046946 characters long

Reading 'data/04. Night Parade, The.txt'...
Corpus is now 4645430 characters long

Reading 'data/05. Icewind Dale - Streams of Silver.txt'...
Corpus is now 5270161 characters long

Reading 'data/05. Ring of Winter, The.epub.txt'...
Corpus is now 5820068 characters long

Reading 'data/06. Crypt of the Shadowking.txt'...
Corpus is now 6348996 characters long

Reading 'data/06. Icewind Dale - The Halfling's Gem.txt'...


Corpus is now 81027838 characters long

Reading 'data/2. Lesser Evils.txt'...
Corpus is now 81824044 characters long

Reading 'data/2. Lies of Light.txt'...
Corpus is now 82358630 characters long

Reading 'data/2. Maestro.txt'...
Corpus is now 83060323 characters long

Reading 'data/2. Mistress of the Night.txt'...
Corpus is now 83615697 characters long

Reading 'data/2. Mistshore.txt'...
Corpus is now 84140438 characters long

Reading 'data/2. Murder in Tarsis.txt'...
Corpus is now 84558757 characters long

Reading 'data/2. Oath of Vigilance.txt'...
Corpus is now 85079740 characters long

Reading 'data/2. Obsidian Ridge.txt'...
Corpus is now 85599531 characters long

Reading 'data/2. Paladins, The.txt'...
Corpus is now 85747335 characters long

Reading 'data/2. Pools of Darkness.txt'...
Corpus is now 86283353 characters long

Reading 'data/2. Promise of the Witch-King.txt'...
Corpus is now 86946231 characters long

Reading 'data/2. Restless Shore, The.txt'...
Corpus is now 87476400 ch

Corpus is now 166549224 characters long

Reading 'data/5. Opportunity for Profit, An.txt'...
Corpus is now 166692465 characters long

Reading 'data/5. Prophet of the Dead.txt'...
Corpus is now 167283414 characters long

Reading 'data/5. Sentinel, The.txt'...
Corpus is now 167790472 characters long

Reading 'data/6. Circle of Skulls.txt'...
Corpus is now 168324534 characters long

Reading 'data/6. Conspiracy.txt'...
Corpus is now 168466137 characters long

Reading 'data/6. Devil You Know, The.txt'...
Corpus is now 169531828 characters long

Reading 'data/6. Herald, The.txt'...
Corpus is now 170142609 characters long

Reading 'data/6. Resurrection.txt'...
Corpus is now 170779868 characters long

Reading 'data/6. Sands of the Soul.txt'...
Corpus is now 171309960 characters long

Reading 'data/6. Simbul's Gift, The.txt'...
Corpus is now 171952766 characters long

Reading 'data/7. Lord of Stormweather.txt'...
Corpus is now 172486710 characters long

Reading 'data/7. Uneasy Alliances.txt'...

**Split the corpus into sentences**

In [10]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [11]:
raw_sentences = tokenizer.tokenize(corpus_raw)

In [12]:
#convert into a list of words
#rtemove unnnecessary,, split into words, no hyphens
#list of words
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

In [13]:
#sentence where each word is tokenized
sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

In [14]:
print(raw_sentences[5])
print(sentence_to_wordlist(raw_sentences[5]))

“The Spider Queen will not accept the sacrifice until the child is named!”
“Drizzt,” breathed Matron Malice.
['The', 'Spider', 'Queen', 'will', 'not', 'accept', 'the', 'sacrifice', 'until', 'the', 'child', 'is', 'named', 'Drizzt', 'breathed', 'Matron', 'Malice']


In [15]:
token_count = sum([len(sentence) for sentence in sentences])
print("The book corpus contains {0:,} tokens".format(token_count))

The book corpus contains 34,603,549 tokens


## Train Word2Vec

In [16]:
# Dimensionality of the resulting word vectors.
num_features = 300

# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
seed = 1

In [17]:
realms2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [18]:
realms2vec.build_vocab(sentences)

2018-02-10 11:35:27,439 : INFO : collecting all words and their counts
2018-02-10 11:35:27,440 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-02-10 11:35:27,472 : INFO : PROGRESS: at sentence #10000, processed 176049 words, keeping 10936 word types
2018-02-10 11:35:27,500 : INFO : PROGRESS: at sentence #20000, processed 344275 words, keeping 15435 word types
2018-02-10 11:35:27,528 : INFO : PROGRESS: at sentence #30000, processed 514416 words, keeping 18487 word types
2018-02-10 11:35:27,556 : INFO : PROGRESS: at sentence #40000, processed 670013 words, keeping 20890 word types
2018-02-10 11:35:27,586 : INFO : PROGRESS: at sentence #50000, processed 843704 words, keeping 22976 word types
2018-02-10 11:35:27,615 : INFO : PROGRESS: at sentence #60000, processed 1015088 words, keeping 24993 word types
2018-02-10 11:35:27,642 : INFO : PROGRESS: at sentence #70000, processed 1169316 words, keeping 26324 word types
2018-02-10 11:35:27,675 : INFO : PROGRESS: a

2018-02-10 11:35:29,619 : INFO : PROGRESS: at sentence #720000, processed 12358516 words, keeping 76982 word types
2018-02-10 11:35:29,648 : INFO : PROGRESS: at sentence #730000, processed 12515454 words, keeping 77358 word types
2018-02-10 11:35:29,676 : INFO : PROGRESS: at sentence #740000, processed 12672407 words, keeping 77747 word types
2018-02-10 11:35:29,713 : INFO : PROGRESS: at sentence #750000, processed 12862618 words, keeping 77980 word types
2018-02-10 11:35:29,742 : INFO : PROGRESS: at sentence #760000, processed 13034647 words, keeping 78095 word types
2018-02-10 11:35:29,783 : INFO : PROGRESS: at sentence #770000, processed 13235042 words, keeping 78735 word types
2018-02-10 11:35:29,817 : INFO : PROGRESS: at sentence #780000, processed 13423758 words, keeping 79324 word types
2018-02-10 11:35:29,842 : INFO : PROGRESS: at sentence #790000, processed 13565492 words, keeping 79750 word types
2018-02-10 11:35:29,869 : INFO : PROGRESS: at sentence #800000, processed 137174

2018-02-10 11:35:31,753 : INFO : PROGRESS: at sentence #1430000, processed 23999233 words, keeping 98519 word types
2018-02-10 11:35:31,782 : INFO : PROGRESS: at sentence #1440000, processed 24157824 words, keeping 99067 word types
2018-02-10 11:35:31,813 : INFO : PROGRESS: at sentence #1450000, processed 24335257 words, keeping 99663 word types
2018-02-10 11:35:31,850 : INFO : PROGRESS: at sentence #1460000, processed 24555264 words, keeping 100435 word types
2018-02-10 11:35:31,884 : INFO : PROGRESS: at sentence #1470000, processed 24748650 words, keeping 101156 word types
2018-02-10 11:35:31,916 : INFO : PROGRESS: at sentence #1480000, processed 24926662 words, keeping 101374 word types
2018-02-10 11:35:31,941 : INFO : PROGRESS: at sentence #1490000, processed 25078446 words, keeping 101608 word types
2018-02-10 11:35:31,972 : INFO : PROGRESS: at sentence #1500000, processed 25242277 words, keeping 101818 word types
2018-02-10 11:35:32,005 : INFO : PROGRESS: at sentence #1510000, pr

2018-02-10 11:35:33,988 : INFO : sample=0.001 downsamples 46 most-common words
2018-02-10 11:35:33,989 : INFO : downsampling leaves estimated 27300027 word corpus (79.0% of prior 34539456)
2018-02-10 11:35:33,989 : INFO : estimated required memory for 71128 words and 300 dimensions: 206271200 bytes
2018-02-10 11:35:34,185 : INFO : resetting layer weights


In [19]:
print("Word2Vec vocabulary length:", len(realms2vec.wv.vocab))

Word2Vec vocabulary length: 71128


**Start training, this might take a minute or two...**

In [20]:
realms2vec.train(sentences, total_words=token_count, epochs=100)

2018-02-10 11:36:22,491 : INFO : training model with 8 workers on 71128 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=7
2018-02-10 11:36:23,513 : INFO : PROGRESS: at 0.01% words, 358333 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:36:24,522 : INFO : PROGRESS: at 0.03% words, 395834 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:36:25,572 : INFO : PROGRESS: at 0.05% words, 403579 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:36:26,586 : INFO : PROGRESS: at 0.06% words, 407652 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:36:27,607 : INFO : PROGRESS: at 0.08% words, 407226 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:36:28,620 : INFO : PROGRESS: at 0.09% words, 412041 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:36:29,631 : INFO : PROGRESS: at 0.11% words, 413799 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:36:30,634 : INFO : PROGRESS: at 0.12% words, 416101 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:36:31,650 : INFO : PROGRESS: at 0.14

2018-02-10 11:37:46,020 : INFO : PROGRESS: at 1.30% words, 426185 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:37:47,027 : INFO : PROGRESS: at 1.32% words, 426036 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:37:48,037 : INFO : PROGRESS: at 1.33% words, 425597 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:37:49,062 : INFO : PROGRESS: at 1.35% words, 424835 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:37:50,078 : INFO : PROGRESS: at 1.36% words, 424326 words/s, in_qsize 14, out_qsize 1
2018-02-10 11:37:51,087 : INFO : PROGRESS: at 1.38% words, 424032 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:37:52,120 : INFO : PROGRESS: at 1.39% words, 423719 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:37:53,124 : INFO : PROGRESS: at 1.41% words, 423963 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:37:54,133 : INFO : PROGRESS: at 1.42% words, 423770 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:37:55,143 : INFO : PROGRESS: at 1.44% words, 423674 words/s, in_qsize 15, out_qsize 0


2018-02-10 11:39:09,453 : INFO : PROGRESS: at 2.56% words, 418125 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:39:10,501 : INFO : PROGRESS: at 2.57% words, 418133 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:39:11,526 : INFO : PROGRESS: at 2.59% words, 418208 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:39:12,539 : INFO : PROGRESS: at 2.61% words, 418347 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:39:13,563 : INFO : PROGRESS: at 2.62% words, 418241 words/s, in_qsize 16, out_qsize 1
2018-02-10 11:39:14,571 : INFO : PROGRESS: at 2.64% words, 418354 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:39:15,587 : INFO : PROGRESS: at 2.65% words, 418541 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:39:16,588 : INFO : PROGRESS: at 2.67% words, 418617 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:39:17,593 : INFO : PROGRESS: at 2.69% words, 418714 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:39:18,595 : INFO : PROGRESS: at 2.70% words, 418806 words/s, in_qsize 15, out_qsize 0


2018-02-10 11:40:32,866 : INFO : PROGRESS: at 3.87% words, 421664 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:40:33,882 : INFO : PROGRESS: at 3.88% words, 421719 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:40:34,890 : INFO : PROGRESS: at 3.90% words, 421791 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:40:35,892 : INFO : PROGRESS: at 3.92% words, 421887 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:40:36,894 : INFO : PROGRESS: at 3.93% words, 421974 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:40:37,899 : INFO : PROGRESS: at 3.95% words, 422000 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:40:38,906 : INFO : PROGRESS: at 3.96% words, 421964 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:40:39,910 : INFO : PROGRESS: at 3.98% words, 421994 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:40:40,918 : INFO : PROGRESS: at 3.99% words, 421963 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:40:41,950 : INFO : PROGRESS: at 4.01% words, 421988 words/s, in_qsize 15, out_qsize 0


2018-02-10 11:41:56,327 : INFO : PROGRESS: at 5.24% words, 428773 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:41:57,331 : INFO : PROGRESS: at 5.26% words, 429008 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:41:58,344 : INFO : PROGRESS: at 5.28% words, 429188 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:41:59,361 : INFO : PROGRESS: at 5.30% words, 429241 words/s, in_qsize 16, out_qsize 2
2018-02-10 11:42:00,386 : INFO : PROGRESS: at 5.32% words, 429436 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:42:01,390 : INFO : PROGRESS: at 5.33% words, 429552 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:42:02,410 : INFO : PROGRESS: at 5.35% words, 429746 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:42:03,418 : INFO : PROGRESS: at 5.37% words, 429912 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:42:04,447 : INFO : PROGRESS: at 5.39% words, 430019 words/s, in_qsize 15, out_qsize 0
2018-02-10 11:42:05,452 : INFO : PROGRESS: at 5.41% words, 430274 words/s, in_qsize 15, out_qsize 0


...

2018-02-10 13:05:06,910 : INFO : PROGRESS: at 94.49% words, 484463 words/s, in_qsize 14, out_qsize 1
2018-02-10 13:05:07,927 : INFO : PROGRESS: at 94.50% words, 484461 words/s, in_qsize 16, out_qsize 0
2018-02-10 13:05:08,928 : INFO : PROGRESS: at 94.52% words, 484463 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:05:09,938 : INFO : PROGRESS: at 94.54% words, 484458 words/s, in_qsize 14, out_qsize 1
2018-02-10 13:05:10,941 : INFO : PROGRESS: at 94.56% words, 484459 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:05:11,960 : INFO : PROGRESS: at 94.57% words, 484455 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:05:12,973 : INFO : PROGRESS: at 94.59% words, 484454 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:05:13,977 : INFO : PROGRESS: at 94.61% words, 484456 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:05:15,008 : INFO : PROGRESS: at 94.63% words, 484448 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:05:16,033 : INFO : PROGRESS: at 94.65% words, 484450 words/s, in_qsize 15, ou

2018-02-10 13:06:30,023 : INFO : PROGRESS: at 95.95% words, 484400 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:06:31,023 : INFO : PROGRESS: at 95.97% words, 484396 words/s, in_qsize 13, out_qsize 2
2018-02-10 13:06:32,036 : INFO : PROGRESS: at 95.98% words, 484387 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:06:33,084 : INFO : PROGRESS: at 96.00% words, 484384 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:06:34,104 : INFO : PROGRESS: at 96.02% words, 484385 words/s, in_qsize 16, out_qsize 1
2018-02-10 13:06:35,114 : INFO : PROGRESS: at 96.04% words, 484387 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:06:36,117 : INFO : PROGRESS: at 96.05% words, 484386 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:06:37,122 : INFO : PROGRESS: at 96.07% words, 484383 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:06:38,146 : INFO : PROGRESS: at 96.09% words, 484378 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:06:39,160 : INFO : PROGRESS: at 96.10% words, 484370 words/s, in_qsize 15, ou

2018-02-10 13:07:53,174 : INFO : PROGRESS: at 97.38% words, 484177 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:07:54,204 : INFO : PROGRESS: at 97.40% words, 484180 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:07:55,210 : INFO : PROGRESS: at 97.42% words, 484183 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:07:56,226 : INFO : PROGRESS: at 97.43% words, 484182 words/s, in_qsize 16, out_qsize 1
2018-02-10 13:07:57,241 : INFO : PROGRESS: at 97.45% words, 484184 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:07:58,250 : INFO : PROGRESS: at 97.47% words, 484178 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:07:59,272 : INFO : PROGRESS: at 97.49% words, 484170 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:08:00,287 : INFO : PROGRESS: at 97.50% words, 484170 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:08:01,300 : INFO : PROGRESS: at 97.52% words, 484166 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:08:02,304 : INFO : PROGRESS: at 97.54% words, 484162 words/s, in_qsize 15, ou

2018-02-10 13:09:16,478 : INFO : PROGRESS: at 98.82% words, 484017 words/s, in_qsize 16, out_qsize 0
2018-02-10 13:09:17,493 : INFO : PROGRESS: at 98.84% words, 484018 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:09:18,531 : INFO : PROGRESS: at 98.86% words, 484014 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:09:19,532 : INFO : PROGRESS: at 98.88% words, 484011 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:09:20,554 : INFO : PROGRESS: at 98.89% words, 484011 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:09:21,570 : INFO : PROGRESS: at 98.91% words, 484015 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:09:22,574 : INFO : PROGRESS: at 98.93% words, 484012 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:09:23,592 : INFO : PROGRESS: at 98.95% words, 484010 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:09:24,601 : INFO : PROGRESS: at 98.97% words, 484004 words/s, in_qsize 15, out_qsize 0
2018-02-10 13:09:25,616 : INFO : PROGRESS: at 98.98% words, 484003 words/s, in_qsize 15, ou

2730008525

**Save to file, can be useful later**

In [21]:
if not os.path.exists("trained"):
    os.makedirs("trained")

In [23]:
realms2vec.save(os.path.join("trained", "realms2vec.w2v"))

2018-02-10 13:12:25,028 : INFO : saving Word2Vec object under trained/realms2vec.w2v, separately None
2018-02-10 13:12:25,028 : INFO : storing np array 'syn0' to trained/realms2vec.w2v.wv.syn0.npy
2018-02-10 13:12:25,089 : INFO : not storing attribute syn0norm
2018-02-10 13:12:25,090 : INFO : storing np array 'syn1neg' to trained/realms2vec.w2v.syn1neg.npy
2018-02-10 13:12:25,126 : INFO : not storing attribute cum_table
2018-02-10 13:12:25,247 : INFO : saved trained/realms2vec.w2v


## Explore the trained model.

In [7]:
realms2vec = w2v.Word2Vec.load(os.path.join("trained", "realms2vec.w2v"))

2018-02-10 13:31:16,234 : INFO : loading Word2Vec object from trained/realms2vec.w2v
2018-02-10 13:31:16,374 : INFO : loading wv recursively from trained/realms2vec.w2v.wv.* with mmap=None
2018-02-10 13:31:16,375 : INFO : loading syn0 from trained/realms2vec.w2v.wv.syn0.npy with mmap=None
2018-02-10 13:31:16,529 : INFO : setting ignored attribute syn0norm to None
2018-02-10 13:31:16,532 : INFO : loading syn1neg from trained/realms2vec.w2v.syn1neg.npy with mmap=None
2018-02-10 13:31:16,679 : INFO : setting ignored attribute cum_table to None
2018-02-10 13:31:16,681 : INFO : loaded trained/realms2vec.w2v


### Compress the word vectors into 2D space and plot them

In [52]:
tsne = sklearn.manifold.TSNE(n_components=2, random_state=0)

In [10]:
all_word_vectors_matrix = realms2vec.wv.syn0

**Train t-SNE, this could take a minute or two...**

NOTE: in my environment this never completes (propably too large). If so, you can skip to "Explore semantic similarities between book characters" section

In [ ]:
all_word_vectors_matrix_2d = tsne.fit_transform(all_word_vectors_matrix)

**Plot the big picture**

In [ ]:
points = pd.DataFrame(
    [
        (word, coords[0], coords[1])
        for word, coords in [
            (word, all_word_vectors_matrix_2d[realms2vec.vocab[word].index])
            for word in realms2vec.vocab
        ]
    ],
    columns=["word", "x", "y"]
)

In [ ]:
points.head(10)

In [ ]:
sns.set_context("poster")

In [ ]:
points.plot.scatter("x", "y", s=10, figsize=(20, 12))

**Zoom in to some interesting places**

In [ ]:
def plot_region(x_bounds, y_bounds):
    slice = points[
        (x_bounds[0] <= points.x) &
        (points.x <= x_bounds[1]) & 
        (y_bounds[0] <= points.y) &
        (points.y <= y_bounds[1])
    ]
    
    ax = slice.plot.scatter("x", "y", s=35, figsize=(10, 8))
    for i, point in slice.iterrows():
        ax.text(point.x + 0.005, point.y + 0.005, point.word, fontsize=11)

**People related to Kingsguard ended up together**

In [ ]:
plot_region(x_bounds=(4.0, 4.2), y_bounds=(-0.5, -0.1))

**Food products are grouped nicely as well. Aerys (The Mad King) being close to "roasted" also looks sadly correct**

In [ ]:
plot_region(x_bounds=(0, 1), y_bounds=(4, 4.5))

### Explore semantic similarities between book characters

**Words closest to the given word**

In [45]:
realms2vec.wv.most_similar("Elminster")

[('El', 0.7434680461883545),
 ('Shadowdale', 0.6829797029495239),
 ('Shandril', 0.6622568368911743),
 ('Mirt', 0.6510697603225708),
 ('Storm', 0.6506309509277344),
 ('Manshoon', 0.6389539837837219),
 ('Mage', 0.6381931304931641),
 ('Sharantyr', 0.6207765340805054),
 ('Narm', 0.6157053709030151),
 ('Srinshee', 0.6120926141738892)]

In [46]:
realms2vec.wv.most_similar("spell")

[('spells', 0.7400574684143066),
 ('enchantment', 0.6742175817489624),
 ('incantation', 0.6424131989479065),
 ('magic', 0.6384660601615906),
 ('casting', 0.5918986797332764),
 ('cast', 0.5778337717056274),
 ('counterspell', 0.5731417536735535),
 ('magics', 0.5412431359291077),
 ('divination', 0.5345152616500854),
 ('charm', 0.5302313566207886)]

In [47]:
realms2vec.wv.most_similar("Amn")

[('Sembia', 0.47273480892181396),
 ('Baldur', 0.462149053812027),
 ('Waterdeep', 0.4475569725036621),
 ('Cormyr', 0.4439469277858734),
 ('Esmeltaran', 0.4284575879573822),
 ('Suzail', 0.4167151153087616),
 ('Tethyr', 0.41307923197746277),
 ('Keczulla', 0.4013843834400177),
 ('Calimshan', 0.396533340215683),
 ('Gate', 0.38300296664237976)]

**Linear relationships between word pairs**

In [48]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = realms2vec.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{start1} is related to {end1}, as {start2} is related to {end2}".format(**locals()))
    return start2

In [49]:
nearest_similarity_cosmul("human", "Elminster", "Drizzt")
nearest_similarity_cosmul("Menzoberranzan", "Drizzt", "Bruenor")
nearest_similarity_cosmul("wizard", "human", "orc")

human is related to Elminster, as drow is related to Drizzt
Menzoberranzan is related to Drizzt, as Mithral is related to Bruenor
wizard is related to human, as shaman is related to orc


'shaman'

In [50]:
realms2vec.wv.doesnt_match("Alustriel Dove Storm Lolth".split()) # Who is not a Silverhand (Seven sister)

'Lolth'

In [51]:
realms2vec.wv.doesnt_match("Lolth Lathander Bahamut Helm Bruenor".split()) # Who is not a god

'Bruenor'